# Z-Score Normalization

In [ ]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split

DATA_PATH = './data'
ratings = pd.read_csv(f'{DATA_PATH}/ratings.csv')
movies  = pd.read_csv(f'{DATA_PATH}/movies.csv')

SEED_LIST = [10, 21, 35, 42, 57]
PERSONA_HEAVY_USER = 414
PERSONA_GENRE_SPECIALIST = 85
TEST_SAMPLE_SIZE = 3000

## 1. 전처리 함수 (Z-Score 정규화)

In [ ]:
def run_preprocessing(train_df, test_df):
    """
    Z-Score 정규화를 통한 데이터 전처리
    
    Args:
        train_df: 학습 데이터프레임
        test_df: 테스트 데이터프레임
    
    Returns:
        train: 전처리된 학습 데이터
        test: 전처리된 테스트 데이터
        user_stats: 사용자별 통계량 (mean, std)
    """
    train = train_df.copy()
    test  = test_df.copy()

    user_stats = train.groupby('userId')['rating'].agg(['mean', 'std']).fillna(1.0)
    user_stats['std'] = user_stats['std'].replace(0, 1.0)

    train = train.merge(user_stats, on='userId', suffixes=('', '_user'))
    train['z_rating'] = (train['rating'] - train['mean']) / train['std']

    test = test.merge(user_stats, on='userId', how='left')
    test = test.dropna(subset=['mean', 'std'])
    test['z_rating'] = (test['rating'] - test['mean']) / test['std']

    return train, test, user_stats

## 2. 평가 함수

In [ ]:
def fast_predict_matrix_no_k(train_df, use_pearson=False):
    """
    양수 유사도를 가진 모든 아이템을 사용하여 예측 행렬 계산
    
    Args:
        train_df: 학습 데이터프레임 (z_rating 포함)
        use_pearson: True면 피어슨 상관계수, False면 코사인 유사도 사용
    
    Returns:
        pred_z: 예측된 Z-score 행렬
        user_ids: 사용자 ID 인덱스
        movie_ids: 영화 ID 인덱스
        valid_mask: 예측 가능 여부 플래그
    """
    pivot_df = train_df.pivot(index='userId', columns='movieId', values='z_rating').fillna(0)
    R = pivot_df.values
    user_ids = pivot_df.index
    movie_ids = pivot_df.columns
    
    if use_pearson:
        item_means = np.mean(R.T, axis=1, keepdims=True)
        R_centered = R.T - item_means
        item_norms = np.linalg.norm(R_centered, axis=1, keepdims=True)
        item_norms = np.where(item_norms == 0, 1.0, item_norms)
        sim_matrix = np.dot(R_centered, R_centered.T) / (item_norms @ item_norms.T)
    else:
        item_norms = np.linalg.norm(R.T, axis=1, keepdims=True)
        item_norms = np.where(item_norms == 0, 1.0, item_norms)
        sim_matrix = np.dot(R.T, R) / (item_norms @ item_norms.T)
    
    np.fill_diagonal(sim_matrix, 0)
    S = np.where(sim_matrix > 0, sim_matrix, 0)

    numerator = np.dot(R, S)
    R_binary = (R != 0).astype(float)
    denominator = np.dot(R_binary, np.abs(S))
    
    with np.errstate(divide='ignore', invalid='ignore'):
        pred_z = numerator / denominator
        valid_mask = (denominator > 0) & np.isfinite(pred_z)
        pred_z = np.where(valid_mask, pred_z, 0.0)
        
    return pred_z, user_ids, movie_ids, valid_mask

In [10]:
def rmse(y_true, y_pred): return np.sqrt(np.mean((np.array(y_true)-np.array(y_pred))**2))
def mae(y_true, y_pred):  return np.mean(np.abs(np.array(y_true)-np.array(y_pred)))

def time_predictions(fn, pairs):
    s = time.perf_counter()
    preds = [fn(u,i) for u,i in pairs]
    e = time.perf_counter() - s
    return preds, e, (e/len(pairs))*1000

## 3. Multi-seed 실험 루프

In [ ]:
results = []
best_reco = None

print(f"실험 시작: 총 {len(SEED_LIST)}개의 시드 테스트.")
print(f"설정: TEST_SAMPLE_SIZE={TEST_SAMPLE_SIZE}")

for idx, SEED in enumerate(SEED_LIST):
    print(f"\nRunning Seed {SEED} ({idx+1}/{len(SEED_LIST)})")

    train_raw, test_raw = train_test_split(ratings, test_size=0.2, random_state=SEED)
    train, test, user_stats = run_preprocessing(train_raw, test_raw)

    s = time.perf_counter()
    pred_matrix_z, u_ids, m_ids, valid_mask = fast_predict_matrix_no_k(train)
    e = time.perf_counter() - s
    
    u_map = {u: i for i, u in enumerate(u_ids)}
    m_map = {m: i for i, m in enumerate(m_ids)}

    test_sample = test[['userId', 'movieId', 'rating', 'mean', 'std']].sample(
        n=min(TEST_SAMPLE_SIZE, len(test)), 
        random_state=SEED
    )

    z_preds = []
    valid_indices = []
    missing_user_count = 0
    missing_movie_count = 0
    nan_inf_count = 0
    fallback_count = 0
    
    user_means = train.groupby('userId')['z_rating'].mean().to_dict()
    global_mean = train['z_rating'].mean()
    
    for idx, (uid, mid) in enumerate(zip(test_sample['userId'], test_sample['movieId'])):
        if uid not in u_map:
            z_preds.append(global_mean)
            missing_user_count += 1
            valid_indices.append(idx)
        elif mid not in m_map:
            fallback_z = user_means.get(uid, global_mean)
            z_preds.append(fallback_z)
            missing_movie_count += 1
            valid_indices.append(idx)
        else:
            u_idx = u_map[uid]
            m_idx = m_map[mid]
            pred_value = pred_matrix_z[u_idx, m_idx]
            is_valid = valid_mask[u_idx, m_idx]
            
            if np.isfinite(pred_value) and is_valid:
                z_preds.append(pred_value)
                valid_indices.append(idx)
            else:
                fallback_z = user_means.get(uid, global_mean)
                z_preds.append(fallback_z)
                nan_inf_count += 1
                fallback_count += 1
                valid_indices.append(idx)
    
    total_missing = missing_user_count + missing_movie_count + nan_inf_count
    if total_missing > 0:
        print(f"  정보: {total_missing}개의 케이스가 예측 불가능합니다 (폴백 전략 적용).")
        if missing_user_count > 0:
            print(f"    ⚠️ Train에 없는 사용자: {missing_user_count}개 (글로벌 평균 사용)")
        if missing_movie_count > 0:
            print(f"    ❗ Train에 없는 영화 (Cold-start): {missing_movie_count}개 (사용자 평균 사용)")
        if nan_inf_count > 0:
            print(f"    ❗예측 불가능 (유사도 없음): {nan_inf_count}개 (사용자 평균 사용)")
    
    original_preds = []
    test_ratings_filtered = []
    for idx in valid_indices:
        pred_z = z_preds[idx]
        row = test_sample.iloc[idx]
        mean_u = row['mean']
        std_u = row['std']
        original_preds.append(pred_z * std_u + mean_u)
        test_ratings_filtered.append(row['rating'])

    total_predictions = pred_matrix_z.size
    avg_ms_per_prediction = (e / total_predictions) * 1000
    avg_ms_per_sample = (e / len(test_sample)) * 1000
    
    res = {
        'seed': SEED,
        'rmse': rmse(test_ratings_filtered, original_preds),
        'mae' : mae(test_ratings_filtered, original_preds),
        'avg_ms': avg_ms_per_sample,
        'avg_ms_per_prediction': avg_ms_per_prediction,
        'matrix_computation_time': e,
        'fallback_count': fallback_count
    }
    results.append(res)
    print(f" -> RMSE: {res['rmse']:.4f}, MAE: {res['mae']:.4f}, Time: {e:.4f}s")

    if idx == 0:
        def recommend(user_id, topk=3):
            """사용자에게 Top-K 영화 추천"""
            if user_id not in u_map:
                return None
            
            try:
                u_idx = u_map[user_id]
                user_row_preds = pred_matrix_z[u_idx]
                user_valid_mask = valid_mask[u_idx]
                watched = set(train[train['userId']==user_id]['movieId'])
                
                user_mean_z = user_stats.loc[user_id, 'mean']
                user_std = user_stats.loc[user_id, 'std']
                
                candidates = []
                for m_idx, m_id in enumerate(m_ids):
                    if m_id in watched: 
                        continue
                    
                    z = user_row_preds[m_idx]
                    is_valid = user_valid_mask[m_idx]
                    
                    if not (np.isfinite(z) and is_valid):
                        z = 0.0
                    
                    final_score = z * user_std + user_mean_z
                    
                    if np.isfinite(final_score):
                        candidates.append((m_id, final_score, is_valid))
                
                candidates.sort(key=lambda x: (-x[2], -x[1]))
                top = candidates[:topk]
                
                recommendations = []
                for m, score, is_valid_pred in top:
                    try:
                        movie_title = movies.loc[movies['movieId']==m, 'title']
                        title = movie_title.iloc[0] if len(movie_title) > 0 else "Unknown"
                    except (IndexError, KeyError):
                        title = "Unknown"
                    
                    recommendations.append({
                        'movieId': m,
                        'title': title,
                        'predicted_rating': round(score, 3),
                        'is_valid_prediction': bool(is_valid_pred)
                    })
                
                return recommendations
            except Exception as e:
                print(f"  경고: 사용자 {user_id} 추천 생성 중 오류: {e}")
                return None

        print("\n[추천 결과 생성 중...]")
        best_reco = {
            'heavy': recommend(PERSONA_HEAVY_USER),
            'specialist': recommend(PERSONA_GENRE_SPECIALIST)
        }

print("\n실험 완료! (결과 요약 셀을 실행하세요)")

실험 시작: K 제한 없이 모든 양수 유사도 사용. 총 5개의 시드 테스트.
설정: TEST_SAMPLE_SIZE=3000

Running Seed 10 (1/5)
  정보: 126개의 케이스가 예측 불가능합니다 (폴백 전략 적용).
    ❗ Train에 없는 영화 (Cold-start): 123개 (사용자 평균 사용)
    ❗예측 불가능 (유사도 없음): 3개 (사용자 평균 사용)
 -> RMSE: 0.8566, MAE: 0.6490, Time: 4.3529s

Running Seed 21 (2/5)
  정보: 117개의 케이스가 예측 불가능합니다 (폴백 전략 적용).
    ❗ Train에 없는 영화 (Cold-start): 117개 (사용자 평균 사용)
 -> RMSE: 0.8639, MAE: 0.6587, Time: 2.9898s

Running Seed 35 (3/5)
  정보: 109개의 케이스가 예측 불가능합니다 (폴백 전략 적용).
    ❗ Train에 없는 영화 (Cold-start): 109개 (사용자 평균 사용)
 -> RMSE: 0.8689, MAE: 0.6642, Time: 3.1073s

Running Seed 42 (4/5)
  정보: 117개의 케이스가 예측 불가능합니다 (폴백 전략 적용).
    ❗ Train에 없는 영화 (Cold-start): 114개 (사용자 평균 사용)
    ❗예측 불가능 (유사도 없음): 3개 (사용자 평균 사용)
 -> RMSE: 0.8829, MAE: 0.6697, Time: 3.3689s

Running Seed 57 (5/5)
  정보: 121개의 케이스가 예측 불가능합니다 (폴백 전략 적용).
    ❗ Train에 없는 영화 (Cold-start): 120개 (사용자 평균 사용)
    ❗예측 불가능 (유사도 없음): 1개 (사용자 평균 사용)
 -> RMSE: 0.8361, MAE: 0.6392, Time: 4.1822s

실험 완료! (결과 요약 셀을 실행하세요)


## 4. 결과 요약

In [12]:
df = pd.DataFrame(results)
print("=== 실험 결과 요약 ===")
display_cols = ['seed','rmse','mae','avg_ms']
if 'fallback_count' in df.columns:
    display_cols.append('fallback_count')
print(df[display_cols].round(4))
print("\n=== 평균 ± 표준편차 ===")
print(f"RMSE : {df.rmse.mean():.4f} ± {df.rmse.std():.4f}")
print(f"MAE  : {df.mae.mean():.4f} ± {df.mae.std():.4f}")
if 'fallback_count' in df.columns:
    print(f"폴백 사용 케이스 : {df.fallback_count.mean():.1f}개 (± {df.fallback_count.std():.1f})")
print(f"예측 속도 (샘플 기준) : {df.avg_ms.mean():.2f} ms (± {df.avg_ms.std():.2f})")
if 'avg_ms_per_prediction' in df.columns:
    print(f"예측 속도 (전체 예측 기준) : {df.avg_ms_per_prediction.mean():.6f} ms (± {df.avg_ms_per_prediction.std():.6f})")
print(f"행렬 계산 시간 : {df.matrix_computation_time.mean():.4f} s (± {df.matrix_computation_time.std():.4f})")

print("\n=== 페르소나 추천 결과 (Seed 10) ===")
if best_reco and best_reco.get('heavy'):
    print(f"헤비 유저 (userId={PERSONA_HEAVY_USER})")
    for x in best_reco['heavy']:
        valid_mark = "✓" if x.get('is_valid_prediction', True) else "⚠"
        print(f"  {valid_mark} {x['title']}  (예측 평점 {x['predicted_rating']})")
else:
    print(f"헤비 유저 (userId={PERSONA_HEAVY_USER}): 추천 결과 없음")

if best_reco and best_reco.get('specialist'):
    print(f"\n드라마 전문가 (userId={PERSONA_GENRE_SPECIALIST})")
    for x in best_reco['specialist']:
        valid_mark = "✓" if x.get('is_valid_prediction', True) else "⚠"
        print(f"  {valid_mark} {x['title']}  (예측 평점 {x['predicted_rating']})")
else:
    print(f"\n드라마 전문가 (userId={PERSONA_GENRE_SPECIALIST}): 추천 결과 없음")

=== 실험 결과 요약 ===
   seed    rmse     mae  avg_ms  fallback_count
0    10  0.8566  0.6490  1.4510               3
1    21  0.8639  0.6587  0.9966               0
2    35  0.8689  0.6642  1.0358               0
3    42  0.8829  0.6697  1.1230               3
4    57  0.8361  0.6392  1.3941               1

=== 평균 ± 표준편차 ===
RMSE : 0.8617 ± 0.0172
MAE  : 0.6562 ± 0.0122
폴백 사용 케이스 : 1.4개 (± 1.5)
예측 속도 (샘플 기준) : 1.20 ms (± 0.21)
예측 속도 (전체 예측 기준) : 0.000658 ms (± 0.000113)
행렬 계산 시간 : 3.6002 s (± 0.6274)

=== 페르소나 추천 결과 (Seed 10) ===
헤비 유저 (userId=414): 추천 결과 없음

드라마 전문가 (userId=85): 추천 결과 없음
